In [1]:
import numpy as np
import pandas as pd
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from nltk.tokenize import PunktSentenceTokenizer
import time

In [2]:
FastTextSocialNetworkModel.MODEL_PATH = '/content/fasttext-social-network-model.bin'

In [3]:
# Loading data - dataset containing lemmatized texts
names = ['id',
         'headline',
         'description',
         'article_body',
         'all_text',
         'text_lem',
         'date',
         'newspaper',
         'city',
         'md_index',
         'url']

data = pd.read_excel('/content/SV-women-data-lem.xlsx',
                     index_col = 0,
                     names = names)
data.head(5)

,id,headline,description,article_body,all_text,text_lem,date,newspaper,city,md_index,url
0,1,«Женщины для утех» по-прежнему нужны: Япония и...,Во время колонизации Кореи Японией в XX веке к...,Во время колонизации Кореи Японией в XX веке к...,«Женщины для утех» по-прежнему нужны: Япония и...,женщина утеха нужный япония китай борьба время...,2016-01-03,ИА Regnum,Москва,2.312,http://regnum.ru/news/2048238.html
1,2,В Норвегии мигрантов научат не насиловать женщин,Наплыв беженцев из Сирии и других стран Ближне...,\n ...,В Норвегии мигрантов научат не насиловать женщ...,норвегия мигрант научить насиловать беженец си...,2016-01-04,Известия (iz.ru),Москва,1.113,http://izvestia.ru/news/600956
2,3,В Кельне задержали первого подозреваемого в на...,\tПрокуратура Кельна сообщила о задержании пер...,"Сейчас+6˚CСейчас в Санкт-ПетербургеОблачно, Бе...",В Кельне задержали первого подозреваемого в на...,кёльн задержать первый подозревать нападение п...,2016-01-05,Фонтанка (fontanka.ru),Санкт-Петербург,1.186,http://www.fontanka.ru/2016/01/05/077/
3,4,В Дании полицейские научат беженцев вежливому ...,В одной из пяти административных областей Дани...,В одной из пяти административных областей Дани...,В Дании полицейские научат беженцев вежливому ...,дания полицейский научить беженец вежливый обр...,2016-01-05,Российская газета (rg.ru),Москва,1.266,http://www.rg.ru/2016/01/05/keln-site.html
4,5,А каждого Михеля демократично переименуют в Му...,"Наш спецкор Дарья Асламова выясняет, в чем при...",В теплом кафе мой собеседник небрежно скидывае...,А каждого Михеля демократично переименуют в Му...,михель демократично переименовать спецкор дарь...,2016-01-06,Комсомольская правда (kp.ru),Москва,0.960,http://www.kp.ru/daily/26476/3347751/


In [4]:
sent_tokenizer = PunktSentenceTokenizer(data.all_text[0])
word_tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=word_tokenizer)
sa_results = pd.DataFrame(columns=['article_id', 'sentence', 'positive', 'negative', 'neutral', 'speech', 'skip'])

In [5]:
%%time

for i in np.arange(data.all_text.shape[0]):

  sentences = sent_tokenizer.tokenize(data.all_text[i])
  results = model.predict(sentences, k=5)

  positive = []
  negative = []
  neutral = []
  speech = []
  skip = []

  for res in results:
    positive.append(res['positive'])
    negative.append(res['negative'])
    neutral.append(res['neutral'])
    speech.append(res['speech'])
    skip.append(res['skip'])

  res_df = pd.DataFrame({'positive': positive, 'negative': negative, 'neutral': neutral, 'speech':speech,'skip': skip})
  sent_df = pd.Series(sentences, name='sentence')
  id_df = pd.Series(((str(i+1)+' ') * len(sentences)).split(), name='article_id')

  i_res = pd.concat([id_df, sent_df, res_df], axis=1)

  sa_results = pd.concat([sa_results, i_res])

sa_results

CPU times: user 4min, sys: 9.26 s, total: 4min 10s
Wall time: 4min 10s


,article_id,sentence,positive,negative,neutral,speech,skip
0,1,«Женщины для утех» по-прежнему нужны: Япония и...,0.039649,0.125933,0.793116,0.009136,0.073706
1,1,Прошлый век знает много примеров государственн...,0.051855,0.156115,0.453272,0.042098,0.098089
2,1,"Государства вели себя как убийцы, садисты, гра...",0.007587,0.921932,0.327678,0.008072,0.119213
3,1,Но история государственного узаконивания сексу...,0.103759,0.250923,0.515630,0.019729,0.075868
4,1,"И речь идет только о тех, кого «организовывало...",0.051855,0.341593,0.577505,0.007356,0.148057
...,...,...,...,...,...,...,...
2,15342,"Она заявила, что подвергалась фиктивным гинеко...",0.100889,0.152042,0.629785,0.009718,0.063725
3,15342,"Хотя это был крайне неприятный опыт, меня заст...",0.314061,0.422515,0.320831,0.000010,0.065615
4,15342,"Но чего я не могла понять, будучи 16-летней де...",0.034110,0.233716,0.665421,0.005070,0.119213
5,15342,"Также модель отметила, что принимала лекарства...",0.122533,0.156115,0.622469,0.014514,0.043376


In [6]:
sa_results.to_excel('SA-results.xlsx')